<a href="https://colab.research.google.com/github/yutan0565/Wake-up-word-Tensorflow/blob/main/Wake_up_word_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 학습 준비

In [3]:
!pip install python_speech_features
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=3a8c4dd09abfcd7b5a2e868e704aabf75e4bf9cbad5a0b3218410f6f20537c90
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
# MFCC 추출 해주기
# https://blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=sooftware&logNo=221661644808 참고
def get_librosa_mfcc(filepath):
  samplie_rate = 8000
  sig, sr = librosa.core.load(filepath, samplie_rate)
  mfccs = librosa.feature.mfcc(y = sig, sr = sr, hop_length =200, n_mfcc = 40, n_fft = 320 )
  return mfccs

# n_mfcc : 음성데이터를 어느 단위로 쪼갤지 (사람은 20 ~ 40 ms 까지는 음소가 바뀌지 못함), return 될 mfcc의 개수
# n_fft : frame의 length = window size,   잘린 음석이 n_ftt보다 작으면 0으로 Padding 해줌, 
   #n_fft는 winddow size보다 크거나 같아야함
   # n_ftt = 8000 * 0.040 = 320
# hop_legth : window 얼마 만큼씩 움질일 것인가

  # n_mfcc = 40
  # hop_length = 200  # 8000 * 0.040
  # N_FFT = 320    # 8000 * 0.040

# 이거 기준으로 나눈 다음에 Mel값을 뽑아서 Feature로 사용하게 된다.  (  50%는 겹치게 분할을 진행)
# 각각의 frame에 대해서 Hamming Window 적용해서 연속성 맞춰주기 - Default 설정임
# 각 프레임에 대하여 Fourier Transform 적용해서(FFT) 주파서 성분 알아내기
# Mel Filter Banb(삶귀처럼, 주파수 증가할수록 큰 삼각형 filter가 생각다고 생각하기)
# 여기까지하면 Mel-Spectrogram Feature가 추출된다.

# Mel-Spectrogram 을 압축해서 표현해주는 DCT 연산 수행 -> Discrete Cosine Transform


In [ ]:
# 데이터가 저장되어 있는 경로 ( class 별로 묶여 있는 곳)
dataset_path = './custum_dataset'

# 폴더 안에 들어 있는 폴더명(class 이름 확인)
for name in listdir(dataset_path): 
  if isdir( "/".join( [dataset_path,name ])) :  # 폴더가 존재 한다면
    print(name)

# 폴더명 = target ,  target이 들어가 있는 list 생성
target_list = [ name for name in listdir(dataset_path) if isdir("/".join( [dataset_path,name ]))]
target_list.remove('_background_noise_')  # 배경음 제거 해주기
print(target_list)

# filname, 을 쪽 모아서,
filenames = []
y = []
for index, target in enumerate(target_list):
    print('/'.join([dataset_path, target]))  # class 에 맞는 폴더 이름 넣어주기
    filenames.append(listdir('/'.join([dataset_path, target])))
    y.append(np.ones(len(filenames[index])) * index) 

# 하나로 쭉 나열 하기
filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]

# file 모아둔거 한번 섞어 주기
filenames_y = list(zip(filenames, y))
random.shuffle(filenames_y)
filenames, y = zip(*filenames_y)

# train, valid, test 나눠 주기
val_ratio = 0.1
test_ratio = 0.2

val_set_size = int(len(filenames) * val_ratio)
test_set_size = int(len(filenames) * test_ratio)

filenames_val = filenames[:val_set_size]
filenames_test = filenames[val_set_size:(val_set_size + test_set_size)]
filenames_train = filenames[(val_set_size + test_set_size):]

y_val = y[:val_set_size]
y_test = y[val_set_size:(val_set_size + test_set_size)]
y_train = y[(val_set_size + test_set_size):]



In [ ]:
# MFCC 를 만들어 주는데 있어서 필요한 변수
target_list = all_targets
feature_sets_file = 'all_targets_mfcc_sets.npz'
perc_keep_samples = 1.0 # 몇퍼센트의 샘플을 사용 할 것인가 (딱히 필요 없는거 같음))
val_ratio = 0.1
test_ratio = 0.1

# 1초에 8000 개의 샘플이 들어감  # 고주파가 사라짐
# 원형은 유지하지만, samplie의 개수가 줄어들임


# 데이터 전처리 - MFCC

# 모델 학습